<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/grad_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports


In [9]:
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from re import L
import json
from operator import itemgetter
from google.colab import files
import pickle5 as pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf




In [5]:
uploaded = files.upload()


Saving hr(tuples).pkl to hr(tuples).pkl


In [6]:
with open("hr(tuples).pkl", "rb") as fh:
  data = pickle.load(fh)

In [105]:
#data = pd.read_pickle("hr(tuples).pkl")

In [12]:
print(data)

      Participant           Stage       0:0.0  0:0.099791  0:0.199582  \
0              22        STIMULUS   (87, 602)   (87, 602)   (87, 602)   
1              22  QUESTIONNAIRES   (85, 921)   (85, 921)   (85, 921)   
2              22         WASHOUT   (80, 749)   (80, 792)   (80, 792)   
3              22        STIMULUS   (81, 719)   (81, 719)   (81, 719)   
4              22  QUESTIONNAIRES   (83, 702)   (82, 702)   (83, 702)   
...           ...             ...         ...         ...         ...   
1339           64        STIMULUS   (78, 516)   (78, 516)   (78, 516)   
1340           64  QUESTIONNAIRES   (77, 799)   (77, 799)   (77, 816)   
1341           64         WASHOUT    (0, 987)    (0, 987)    (0, 987)   
1342           64        STIMULUS   (69, 829)   (68, 829)   (69, 829)   
1343           64  QUESTIONNAIRES  (66, 1245)  (65, 1245)  (65, 1245)   

     0:0.299373  0:0.399164 0:0.49895500000000004  0:0.598746  \
0     (87, 602)   (87, 602)             (87, 602)   (87, 6

In [25]:
def getter(x):
    if isinstance(x,tuple):
        return itemgetter(0)(x)
    else:
        return x
        
#data = pd.read_pickle("hr(tuples).pkl")
#mask = data["Participant"] ==  22 
mask2 = data["Stage"] == "STIMULUS"
mask3 = data["Target"]== "BASELINE"
#print(data.loc[ mask2,[*data.columns[2:500],*data.columns[-15:]]].applymap(lambda x:getter(x)))
#print(data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x)))#xtrain->particpants hr values
print(data.loc[mask2,"Target"])#ytrain-> target emo


0        BASELINE
3         NEUTRAL
6             AWE
9         DISGUST
12       SURPRISE
          ...    
1330         FEAR
1333    AMUSEMENT
1336      SADNESS
1339      NEUTRAL
1342          AWE
Name: Target, Length: 462, dtype: object


In [26]:
xtrain=data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x))
ytrain=data.loc[mask2,"Target"]

In [68]:
x_train=[]
y_train=[]
label_encoder = LabelEncoder()
#for i in range(6289,1342):
x_train,y_train=np.array(xtrain),np.array(ytrain)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
encoded_y = label_encoder.fit_transform(y_train)
label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                         label_encoder.transform(label_encoder.classes_)))
print(label_encoder_name_mapping)
print(encoded_y)
print(y_train)

{'AMUSEMENT': 0, 'ANGER': 1, 'AWE': 2, 'BASELINE': 3, 'DISGUST': 4, 'ENTHUSIASM': 5, 'FEAR': 6, 'LIKING': 7, 'NEUTRAL': 8, 'SADNESS': 9, 'SURPRISE': 10}
[ 3  8  2  4 10  1  5  7  6  0  9  3  2  4 10  1  5  7  6  0  9  8  3  4
 10  1  5  7  6  0  9  8  2  3 10  1  5  7  6  0  9  8  2  4  3  1  5  7
  6  0  9  8  2  4 10  3  5  7  6  0  9  8  2  4 10  1  3  7  6  0  9  8
  2  4 10  1  5  3  6  0  9  8  2  4 10  1  5  7  3  9  8  2  4 10  1  5
  7  6  0  3  8  2  4 10  1  5  7  6  0  9  3  2  4 10  1  5  7  6  0  9
  8  3  4 10  1  5  7  6  0  9  8  2  3 10  1  5  7  6  0  9  8  2  4  3
  1  5  7  6  0  9  8  2  4 10  3  5  7  6  0  9  8  2  4 10  1  3  7  6
  0  9  8  2  4 10  1  5  3  0  9  8  2  4 10  1  5  7  6  3  6  0  9  8
  2  4 10  1  5  7  3  9  8  2  4 10  1  5  7  6  0  3  8  2  4 10  1  5
  7  6  0  9  3  2  4 10  1  5  7  6  0  9  8  3  4 10  1  5  7  6  0  9
  8  2  3 10  1  5  7  6  0  9  8  2  4  3  1  5  7  6  0  9  8  2  4 10
  3  5  7  6  0  9  8  2  4 10  1  3  7  6  

In [95]:
#initalizing the RNN
regressor =Sequential()

In [96]:
#adding the first lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(Dropout(0.2))

In [97]:
#adding the 2nd lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [98]:
#adding the 3rd lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [99]:
#adding the 4th lstm layer and some dropout regularisation
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

In [100]:
#adding the output layer
regressor.add (Dense(units=11,activation='softmax'))

In [101]:
#compiling the RNN
regressor.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')

In [102]:
print(regressor.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 6290, 50)          10400     
                                                                 
 dropout_12 (Dropout)        (None, 6290, 50)          0         
                                                                 
 lstm_13 (LSTM)              (None, 6290, 50)          20200     
                                                                 
 dropout_13 (Dropout)        (None, 6290, 50)          0         
                                                                 
 lstm_14 (LSTM)              (None, 6290, 50)          20200     
                                                                 
 dropout_14 (Dropout)        (None, 6290, 50)          0         
                                                                 
 lstm_15 (LSTM)              (None, 50)               

In [ ]:
#Fitting the RNN to the training set
regressor.fit(x_train,encoded_y,epochs=100,batch_size=50,callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',
    patience=5,
    restore_best_weights=True)
])

cross validation part for particpants